In [13]:
!pip install sqlalchemy pymysql -q

In [14]:
!pip install sqlalchemy sql_magic -q

In [17]:
%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [47]:
%sql

UsageError: Line magic function `%sql` not found.


In [2]:
import pandas as pd
import numpy as np


## Write pandas df to database

In [3]:
user="cornell"
password="Pitney@1a"
host="gec-dwh.ctz5iw1v2fh7.us-west-2.rds.amazonaws.com"
db="cornell"

conn = f"mysql+pymysql://{user}:{password}@{host}/{db}"

In [4]:
from sqlalchemy import create_engine

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

In [5]:
from pandas.io import sql


## Query database

In [6]:
result = engine.execute('show databases')
for _r in result:
    print(_r)

('information_schema',)
('cornell',)


In [42]:
result = engine.execute('show tables in cornell')
for _r in result:
    print(_r)

('deliver_package_info',)
('input_df',)
('ne_delivery_data_by_day_2018',)
('ne_returns_data_by_day_2018',)
('return_package_info',)
('return_package_info_update',)
('test_df',)


## Query first 10 delivery package data & fields name

In [61]:
result = engine.execute('select * from cornell.ne_delivery_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

(datetime.date(2018, 9, 30), 'Delivery', '362970510', '0.1950', 'Client - 2462', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082836', '0.2300', 'Client - 1216', '11096', 'Courier - Plainville', '2762')
(datetime.date(2018, 8, 14), 'Delivery', '355433187', '1.6600', 'Client - 1216', '11096', 'Courier - Secaucus', '7094')
(datetime.date(2018, 7, 11), 'Delivery', '350299758', '0.8800', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082834', '0.4850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 9, 6), 'Delivery', '358082830', '15.1200', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 9), 'Delivery', '350140996', '1.3400', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(2018, 7, 11), 'Delivery', '350286964', '0.1850', 'Client - 1216', '11096', 'IND - PartnerFacility', '46037')
(datetime.date(20

In [35]:
result = engine.execute('describe cornell.deliver_package_info')
for _r in result:
    print(_r)

('index', 'bigint(20)', 'YES', 'MUL', None, '')
('origindate', 'date', 'YES', '', None, '')
('originfacilityzip', 'text', 'YES', '', None, '')
('destinationfacilityzip', 'text', 'YES', '', None, '')
('count', 'bigint(20)', 'YES', '', None, '')
('origin_major_city', 'text', 'YES', '', None, '')
('origin_county', 'text', 'YES', '', None, '')
('origin_state', 'text', 'YES', '', None, '')
('origin_lat', 'double', 'YES', '', None, '')
('origin_lng', 'double', 'YES', '', None, '')
('dest_major_city', 'text', 'YES', '', None, '')
('dest_county', 'text', 'YES', '', None, '')
('dest_state', 'text', 'YES', '', None, '')
('dest_lat', 'double', 'YES', '', None, '')
('dest_lng', 'double', 'YES', '', None, '')


## How many delivery data do we have

In [10]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018')
for _r in result:
    print(_r)

(61916290,)


## Daily demand of delivery packages

In [ ]:
result = engine.execute('select origindate, count(*) from cornell.ne_delivery_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

### The number of deliver packages from customer warehouse to USPS & store in a new table "deliver_package_info"

In [12]:
# The # of packages from customer warehouse to USPS 
deliver_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_delivery_data_by_day_2018 \
                        group by originfacilityzip,destinationfacilityzip \
                        order by origindate')
deliver_df = pd.DataFrame(deliver_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [14]:
deliver_df.to_sql(con=engine, name='deliver_package_info', 
                if_exists='replace')

## Query first 10 return package data & fields name

In [ ]:
result = engine.execute('select * from cornell.ne_returns_data_by_day_2018 limit 10')
for _r in result:
    print(_r)

In [ ]:
result = engine.execute('describe cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

## How many return data do we have

In [9]:
result = engine.execute('select count(*) from cornell.ne_returns_data_by_day_2018')
for _r in result:
    print(_r)

(33276614,)


## Daily demand of return packages

In [ ]:
result = engine.execute('select origindate, count(*) from cornell.ne_returns_data_by_day_2018 group by origindate')
for _r in result:
    print(_r)

### The number of daily return packages from USPS to customer warehouse & store in a new table "return_package_info"

In [15]:
return_result = engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count(*) from cornell.ne_returns_data_by_day_2018 \
                        group by originfacilityzip,destinationfacilityzip \
                        order by origindate')
return_df = pd.DataFrame(return_result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])




In [17]:
return_df.to_sql(con=engine, name='return_package_info', if_exists='replace')

## geodecode zip3

In [18]:
#!pip install uszipcode

In [20]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

### decodezip3 function

In [21]:
def decodezip3(zip3):
    res=[]
    for i in zip3:
        if not i:
            tmp=['','','','','']
        else:        
            zipcode=search.by_zipcode(i)
            zipinfo=zipcode.to_dict()
            tmp=[zipinfo['major_city'],zipinfo['county'],zipinfo['state'],zipinfo['lat'],zipinfo['lng']]
            res.append(tmp)
    return res


### decode deliver - origin

In [ ]:
deliver_df=engine.execute('select * from deliver_package_info')

In [22]:
deliver_tmp=decodezip3(deliver_df['originfacilityzip'])

In [23]:
tmp_df=pd.DataFrame(deliver_tmp,columns=['origin_major_city','origin_county','origin_state','origin_lat','origin_lng'])

In [24]:
deliver_df=deliver_df.join(tmp_df)

### decode deliver - dest

In [22]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from deliver_package_info')
deliver_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [26]:
dest_df=pd.DataFrame(destination_tmp,columns=['dest_major_city','dest_county','dest_state','dest_lat','dest_lng'])


In [27]:
deliver_df=deliver_df.join(dest_df)

In [28]:
deliver_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_major_city,origin_county,origin_state,origin_lat,origin_lng,dest_major_city,dest_county,dest_state,dest_lat,dest_lng
0,2018-01-01,10974,07097,14,Sloatsburg,Rockland County,NY,41.15,-74.20,Jersey City,Hudson County,NJ,NaN,NaN
1,2018-01-01,21057,24012,8,Glen Arm,Baltimore County,MD,39.46,-76.51,Roanoke,Roanoke city,VA,37.32,-79.90
2,2018-01-01,21057,06492,4,Glen Arm,Baltimore County,MD,39.46,-76.51,Wallingford,New Haven County,CT,41.46,-72.81
3,2018-01-01,21057,12288,2,Glen Arm,Baltimore County,MD,39.46,-76.51,Albany,Albany County,NY,NaN,NaN
4,2018-01-03,21057,3820,4,Glen Arm,Baltimore County,MD,39.46,-76.51,Dover,Strafford County,NH,43.19,-70.89
5,2018-01-03,08837,,2,Edison,Middlesex County,NJ,40.52,-74.34,Endicott,Broome County,NY,42.10,-76.10
6,2018-01-03,06492,,4,Wallingford,New Haven County,CT,41.46,-72.81,Albany,Albany County,NY,42.72,-73.83
7,2018-01-03,10533,13760,6,Irvington,Westchester County,NY,41.04,-73.86,Harrisburg,Dauphin County,PA,40.26,-76.85
8,2018-01-03,11096,,8133,Inwood,Nassau County,NY,40.62,-73.75,Bethpage,Nassau County,NY,40.74,-73.48
9,2018-01-04,06492,12205,1379,Wallingford,New Haven County,CT,41.46,-72.81,Rochester,Monroe County,NY,43.13,-77.74


In [29]:
deliver_df.to_sql(con=engine, name='deliver_package_info', if_exists='replace')

### decode return origin

In [18]:
result=engine.execute('select origindate,originfacilityzip,destinationfacilityzip,count from return_package_info')
return_df = pd.DataFrame(result.fetchall(),columns=['origindate','originfacilityzip','destinationfacilityzip','count'])

In [23]:
return_tmp=decodezip3(return_df['originfacilityzip'])

In [25]:
tmp_df=pd.DataFrame(return_tmp,columns=['origin_major_city','origin_county','origin_state','origin_lat','origin_lng'])

In [26]:
return_df=return_df.join(tmp_df)

### decode return destination

In [28]:
return_tmp=decodezip3(return_df['destinationfacilityzip'])

In [29]:
tmp_df=pd.DataFrame(return_tmp,columns=['dest_major_city','dest_county','dest_state','dest_lat','dest_lng'])

In [30]:
return_df=return_df.join(tmp_df)

In [40]:
return_df

,origindate,originfacilityzip,destinationfacilityzip,count,origin_major_city,origin_county,origin_state,origin_lat,origin_lng,dest_major_city,dest_county,dest_state,dest_lat,dest_lng
0,2018-01-02,15930,27801,2,Dunlo,Cambria County,PA,40.293,-78.722,Rocky Mount,Edgecombe County,NC,35.90,-77.70
1,2018-01-02,23222,07836,1,Richmond,Richmond city,VA,37.580,-77.420,Flanders,Morris County,NJ,40.85,-74.71
2,2018-01-02,4092,07836,4,Westbrook,Cumberland County,ME,43.700,-70.350,Flanders,Morris County,NJ,40.85,-74.71
3,2018-01-02,01472,52404,1,West Groton,Middlesex County,MA,NaN,NaN,Cedar Rapids,Linn County,IA,41.90,-91.70
4,2018-01-02,15930,17699,2,Dunlo,Cambria County,PA,40.293,-78.722,Lancaster,Lancaster County,PA,NaN,NaN
5,2018-01-02,20676,21704,1,Port Republic,Calvert County,MD,38.490,-76.540,Frederick,Frederick County,MD,39.34,-77.38
6,2018-01-02,08880,16368,1,South Bound Brook,Somerset County,NJ,40.550,-74.530,Warren,Warren County,PA,NaN,NaN
7,2018-01-02,20676,46037,1,Port Republic,Calvert County,MD,38.490,-76.540,Fishers,Hamilton County,IN,39.96,-85.95
8,2018-01-02,07721,21704,1,Cliffwood,Monmouth County,NJ,40.440,-74.230,Frederick,Frederick County,MD,39.34,-77.38
9,2018-01-02,15930,30349,1,Dunlo,Cambria County,PA,40.293,-78.722,Atlanta,Fulton County,GA,33.60,-84.50


In [41]:
# return_df.to_sql(con=engine, name='return_package_info', if_exists='replace',dtype=['datetime.date','text','text','bigint(20)','text','text','text','double','double','text','text','text','double','double'])
return_df.to_sql(con=engine, name='return_package_info_update', if_exists='replace')
